## Install Libraries

In [1]:
! pip install --quiet --upgrade chromadb
! pip install --quiet --upgrade pandas
! pip install --quiet --upgrade llama-index
! pip install --quiet --upgrade openai
! pip install --quiet --upgrade llama-index-vector-stores-chroma

## Setup Model
#### Must create environment variable with key https://help.openai.com/en/articles/5112595-best-practices-for-api-key-safety

In [ ]:
import os
import openai

openai.api_key = os.environ["OPENAI_API_KEY"]

## Create Chroma Client

In [3]:
import chromadb
chroma_client = chromadb.Client()


/Users/brody/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


## Create Collection

In [4]:
eventsCollection = chroma_client.create_collection(name="Events")
ostemCollection = chroma_client.create_collection(name="OSTEM")
pathwaysCollection = chroma_client.create_collection(name="Pathways")
researchCollection = chroma_client.create_collection(name="Research")

## Add Documents to Collection

#### Create Dataframes

In [5]:
import pandas as pd

relativePath="Backend/data/"

events = pd.read_csv(f"{relativePath}events.csv")
ostem = pd.read_csv(f"{relativePath}ostems.csv")
pathways = pd.read_csv(f"{relativePath}pathways.csv")
research = pd.read_csv(f"{relativePath}solicitations.csv")

In [6]:
from llama_index.core.schema import TextNode
from llama_index.core import StorageContext
from llama_index.core import VectorStoreIndex
from llama_index.vector_stores.chroma import ChromaVectorStore
from llama_index.embeddings.openai import OpenAIEmbedding


#### Events

In [7]:
idEvents=events["ID"]
titleEvents=events["Title"]
descriptionEvents=events["Description"]
urlEvents=events["URL"]
typeEvents=events["Type"]

print(len(idEvents))

display(events)

9


,ID,Title,Description,URL,Type
0,1,12/4 “FIRST WOMEN Series” Live Q&A: NASA's 202...,12/4 “FIRST WOMEN Series” Live Q&A: NASA's 202...,https://stemgateway.nasa.gov/s/course-offering...,Educator
1,2,Celebrating Native American Heritage Month wit...,Celebrating Native American Heritage Month wit...,https://stemgateway.nasa.gov/s/course-offering...,Student and Educator
2,3,FY25 NCAS Mission 1: Discover B,Please review all the information on the NCAS ...,https://stemgateway.nasa.gov/s/course-offering...,Student
3,4,12/5 Celebrate Computer Science at KSC: Virtua...,Celebrate Computer Science at Kennedy Space Ce...,https://stemgateway.nasa.gov/s/course-offering...,Educator
4,5,12/11 Virtual Classroom Connection: Astro-Not-...,12/11 Virtual Classroom Connection: Astro-Not-...,https://stemgateway.nasa.gov/s/course-offering...,Educator
5,6,Exp 73 NGS In-Flight Education Downlinks,In-Flight Education Downlinks are opportunitie...,https://stemgateway.nasa.gov/s/course-offering...,Educator
6,7,12/12 Surprisingly STEM Live Q&A: Memory Metal...,"Event Date: December 12, 2024, at 4 p.m. ET (T...",https://stemgateway.nasa.gov/s/course-offering...,Educator
7,8,2024-25 NASA STEM Kids Virtual Event Group Reg...,The NASA Glenn Research Center’s Office of Sci...,https://stemgateway.nasa.gov/s/course-offering...,Student
8,9,2024-25 NASA STEM Kids Virtual Event Individua...,The NASA Glenn Research Center’s Office of Sci...,https://stemgateway.nasa.gov/s/course-offering...,Student


In [ ]:
def eventsResponse(dataframe):
    nodes = []
    for _, row in dataframe.iterrows():
        node = TextNode(
            text=row["Description"],
            metadata={
                "id": row["ID"],
                "title": row["Title"],
                "url": row["URL"],
                "type": row["Type"],
            },
        )
        nodes.append(node)

    vector_store = ChromaVectorStore(chroma_collection=eventsCollection)
    storage_context = StorageContext.from_defaults(vector_store=vector_store)
    
    embed_model = OpenAIEmbedding(model="text-embedding-3-small")
    
    index = VectorStoreIndex(nodes, storage_context=storage_context, embed_model=embed_model)
    
    query_engine = index.as_query_engine()
    response = query_engine.query("Events for woman")
    return response

In [19]:
from IPython.display import Markdown

response = eventsResponse(events)
Markdown(f"<b>{response}</b>")

<b>The event mentioned in the context is part of a series called "FIRST WOMEN," which aims to highlight and feature women who are making history at NASA.</b>

#### OSTEM

In [10]:
idOstem=ostem["ID"]
titleOstem=ostem["Title"]
descriptionOstem=ostem["Description"]
urlOstem=ostem["URL"]
typeOstem=ostem["Type"]

display(ostem)

,ID,Title,Description,URL,Type
0,1,12/4 “FIRST WOMEN Series” Live Q&A: NASA's 202...,12/4 “FIRST WOMEN Series” Live Q&A: NASA's 202...,https://stemgateway.nasa.gov/s/course-offering...,Educator
1,2,Celebrating Native American Heritage Month wit...,Celebrating Native American Heritage Month wit...,https://stemgateway.nasa.gov/s/course-offering...,Student and Educator
2,3,FY25 NCAS Mission 1: Discover B,Please review all the information on the NCAS ...,https://stemgateway.nasa.gov/s/course-offering...,Student
3,4,12/5 Celebrate Computer Science at KSC: Virtua...,Celebrate Computer Science at Kennedy Space Ce...,https://stemgateway.nasa.gov/s/course-offering...,Educator
4,5,12/11 Virtual Classroom Connection: Astro-Not-...,12/11 Virtual Classroom Connection: Astro-Not-...,https://stemgateway.nasa.gov/s/course-offering...,Educator
...,...,...,...,...,...
244,245,Dec 12 Lunabotics Junior Focus Group for Onlin...,"Can teaching robotics be done virtually? Yes, ...",https://stemgateway.nasa.gov/s/course-offering...,Educator
245,246,NQuest Grades 6-8 (2024-2025),Are you ready for NASA to inspire your classro...,https://stemgateway.nasa.gov/s/course-offering...,Educator
246,247,Nov 14 Lunabotics Junior Workshop*,REQUIREMENT: MUST BE WITHIN DRIVING DISTANCE O...,https://stemgateway.nasa.gov/s/course-offering...,Educator
247,248,Oct 17 Lunabotics Junior Workshop*,REQUIREMENT: MUST BE WITHIN DRIVING DISTANCE O...,https://stemgateway.nasa.gov/s/course-offering...,Educator


In [ ]:
def ostemResponse(dataframe):
    nodes = []
    for _, row in dataframe.iterrows():
        node = TextNode(
            text=row["Description"],
            metadata={
                "id": row["ID"],
                "title": row["Title"],
                "url": row["URL"],
                "type": row["Type"],
            },
        )
        nodes.append(node)

    vector_store = ChromaVectorStore(chroma_collection=eventsCollection)
    storage_context = StorageContext.from_defaults(vector_store=vector_store)
    
    embed_model = OpenAIEmbedding(model="text-embedding-3-small")
    
    index = VectorStoreIndex(nodes, storage_context=storage_context, embed_model=embed_model)
    
    query_engine = index.as_query_engine()
    response = query_engine.query("Events for woman")
    return response

#### Pathways

In [11]:
idPathways=pathways["ID"]
titlePathways=pathways["Title"]
educationLevel=pathways["Education Level"]
urlPathways=pathways["URL"]
majorsPathways=pathways["Majors"]

display(pathways)

,ID,Title,Education Level,URL,Majors
0,1,Human Resources,Undergraduate and Graduate students,https://nasa.usajobs.gov/job/804519800,"Business, Communications, Human Resources, Ind..."
1,2,Aerospace or Aeronautical Engineering,Undergraduate and Graduate students,https://nasa.usajobs.gov/job/804503500,"Aeronautical Engineering, Aerospace Engineerin..."
2,3,Computer or Software Engineering,Undergraduate and Graduate students,https://nasa.usajobs.gov/job/804519600,"Computer Engineering, Computer Science, Comput..."
3,4,Electrical Engineering,Undergraduate and Graduate students,https://nasa.usajobs.gov/job/804504100,"Electrical Engineering, Electronics Engineerin..."
4,5,Engineering,Doctorate-level students,https://nasa.usajobs.gov/job/804506300,"Aeronautical Engineering, Aerospace Engineerin..."
5,6,Engineering Physics,Undergraduate and Graduate students,https://nasa.usajobs.gov/job/804501500,"Chemical Physics, Engineering Physics, Nuclear..."
6,7,Industrial or Civil Engineering,Undergraduate and Graduate students,https://nasa.usajobs.gov/job/804501100,"Architecture, Civil Engineering, Industrial En..."
7,8,Materials or Chemical Engineering,Undergraduate and Graduate students,https://nasa.usajobs.gov/job/804518600,"Ceramic Engineering, Chemical Engineering, Che..."
8,9,Mechanical Engineering,Undergraduate and Graduate students,https://nasa.usajobs.gov/job/804505800,"Engineering Mechanics, Mechanical Engineering,..."
9,10,Research Engineering,Doctorate-level students,https://nasa.usajobs.gov/job/804500600,"Aerospace/Aeronautical Engineering, Chemistry,..."


#### Research

In [12]:
idResearch=research["ID"]
titleResearch=research["Solicitation Title"]
status=research["Status"]
idResearch=research["Solicitation ID"]
urlResearch=research["URL"]

display(research)

,ID,Solicitation Title,Status,Solicitation ID,URL
0,1,2024 HERO Appendix E: NASA Human Research Prog...,Due in 30 days,80JSC024NA001-FLAGSHIP2,https://nspires.nasaprs.com/external/solicitat...
1,2,2025 HERO Appendix A: NASA Human Research Prog...,Due in 30 days,NNJ25ZSA001N-OMNIBUS,https://nspires.nasaprs.com/external/solicitat...
2,3,A.14 Integrated Water Field Campaign,Due in 30 days,NNH24ZDA001N-WFC,https://nspires.nasaprs.com/external/solicitat...
3,4,A.32 Precipitation Measurements Mission and Cl...,Due in 30 days,NNH24ZDA001N-PMMCCST,https://nspires.nasaprs.com/external/solicitat...
4,5,A.61 INjected Smoke and PYRocumulonimbus Exper...,Due in 30 days,NNH24ZDA001N-INSPYRE,https://nspires.nasaprs.com/external/solicitat...
...,...,...,...,...,...
126,127,Space Technology Research Development Demonstr...,Open,NNH25ZTR001N,https://nspires.nasaprs.com/external/solicitat...
127,128,Topical Workshops Symposiums and Conferences (...,Open,NNH24ZDA002N,https://nspires.nasaprs.com/external/solicitat...
128,129,TRISH Announcement Soliciting Postdoctoral Fel...,Open,TRISHRFA2501PD,https://nspires.nasaprs.com/external/solicitat...
129,130,University Student Research Challenge (USRC),Open,NNH24ZEA001N-USRC,https://nspires.nasaprs.com/external/solicitat...


In [13]:
node = [TextNode(
            text="12/4 “FIRST WOMEN Series” Live Q&A: NASA's 2024 Wings of Excellence Awardee Event Date: December 4, 2024, at 4 p.m. ET (Wednesday) Registration Deadline: November 18, 2024, at 11:59 p.m. ET (Monday) This year we are introducing a new series, 'FIRST WOMEN,' Virtual Classroom Connections. Women are making history at NASA every day and now we're giving you the chance to meet them. This series h...",
            metadata={
                "id": 1,
                "title": "12/4 “FIRST WOMEN Series” Live Q&A: NASA's 2024 Wings of Excellence Awardee",
                "url": "https://stemgateway.nasa.gov/s/course-offering/a0B3d0000025iVkEAI/124-first-women-series-live-qa-nasas-2024-wings-of-excellence-awardee",
                "type": "Educator",
            }
)]

In [14]:
from llama_index.embeddings.openai import OpenAIEmbedding

embed_model = OpenAIEmbedding(
    model="text-embedding-3-small",
    api_key=openai.api_key
)